In [3]:
import pickle

In [5]:
# Path to your pickle file
pickle_file_path = 'data/models/model_education_loan.pkl'

# Load the model
with open(pickle_file_path, 'rb') as file:
    model = pickle.load(file)

/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.2 when using version 1.4.2. This might lead to break

In [6]:
model

Pipeline(steps=[('Vectorizer', DictVectorizer()),
                ('Standardization', StandardScaler(with_mean=False)),
                ('Classifier',
                 MLPClassifier(hidden_layer_sizes=[50, 50, 50, 50, 50, 50]))])

In [9]:
!pip install boto3 botocore


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import numpy as np
import onnx
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.neural_network import MLPClassifier

# Assuming 'model' is your trained MLPClassifier
# Sample input for the shape of the model
initial_type = [('float_input', FloatTensorType([None, 25]))]

# Convert to ONNX format
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
onnx_model_path = "onnx_models/education_loan.onnx"
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())


In [39]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [41]:
import botocore.exceptions
import time

def list_objects(prefix, retries=5):
    try:
        filter = bucket.objects.filter(Prefix=prefix)
        
        # Retry logic with exponential backoff
        for attempt in range(retries):
            try:
                for obj in filter.all():
                    print(obj.key)
                break  # Exit loop if successful
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] == '503':  # Handle 503 errors specifically
                    if attempt < retries - 1:
                        wait_time = 2 ** attempt
                        print(f"List operation failed with error 503. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to list objects after {retries} attempts. Giving up.")
                        raise
                else:
                    raise  # Re-raise the error if it's not a 503
    except botocore.exceptions.ClientError as e:
        print(f"Error listing objects: {e}")


In [42]:
list_objects('model')

List operation failed with error 503. Retrying in 1 seconds...
List operation failed with error 503. Retrying in 2 seconds...
List operation failed with error 503. Retrying in 4 seconds...
List operation failed with error 503. Retrying in 8 seconds...
Failed to list objects after 5 attempts. Giving up.
Error listing objects: An error occurred (503) when calling the ListObjects operation (reached max retries: 4): Service Unavailable


In [16]:
! ls -alRh ./onnx_models/

./onnx_models/:
total 156K
drwxr-sr-x.  2 1009940000 1009940000 4.0K Sep 12 06:52 .
drwxr-sr-x. 11 1009940000 1009940000 4.0K Sep 12 07:03 ..
-rw-r--r--.  1 1009940000 1009940000 145K Sep 12 06:52 education_loan.onnx


In [22]:
pip install openvino-dev[onnx]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 155.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 301.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 203.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 132.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 292.7 MB/s eta 0:00:00
  Attempting uninstall: fastjsonschema
    Found existing installation: fastjsonschema 2.20.0
    Uninstalling fastjsonschema-2.20.0:
      Successfully uninstalled fastjsonschema-2.20.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.4
    Uninstalling protobuf-4.25.4:
      Successfully uninstalled protobuf-4.25.4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninst

In [57]:
!ovc onnx_models/1/education_loan.onnx --output_model /openvino_model/education_loan.xml

[ ERROR ]  -------------------------------------------------
[ ERROR ]  ----------------- INTERNAL ERROR ----------------
[ ERROR ]  Unexpected exception happened.
[ ERROR ]  Please verify parameters and environment.
[ ERROR ]  If you think this is a bug, please create new ticket here: 
[ ERROR ]  https://github.com/openvinotoolkit/openvino/issues.
[ ERROR ]  -------------- DETAILED INFORMATION -------------
[ ERROR ]  Check 'false' failed at src/frontends/onnx/frontend/src/frontend.cpp:144:
FrontEnd API failed with GeneralFailure:
OpenVINO does not support the following ONNX operations: ai.onnx.ml.DictVectorizer, ai.onnx.ml.Scaler, ai.onnx.ml.ZipMap, ai.onnx.ml.ArrayFeatureExtractor

[ ERROR ]  Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/openvino/tools/ovc/convert_impl.py", line 494, in _convert
    ov_model = driver(argv, {"conversion_parameters": non_default_params})
  File "/opt/app-root/lib64/python3.9/site-packages/openvino/tools/ovc/con

In [63]:
pip install onnx skl2onnx onnxmltools


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pickle

# Load the trained model
with open("data/models/model_education_loan.pkl", "rb") as model_file:
    pipe = pickle.load(model_file)

# Print the pipeline steps to inspect their names and structure
print(pipe.named_steps)


{'Vectorizer': DictVectorizer(), 'Standardization': StandardScaler(with_mean=False), 'Classifier': MLPClassifier(hidden_layer_sizes=[50, 50, 50, 50, 50, 50])}


In [67]:
import pickle
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.pipeline import Pipeline

def convert_to_onnx(model_filename, onnx_filename):
    # Load the trained model
    with open(model_filename, "rb") as model_file:
        pipe = pickle.load(model_file)
    
    # Define the initial types based on your feature size
    initial_type = [('input', FloatTensorType([None, len(pipe.named_steps['preprocessor'].transformers_[0][1].named_steps['scaler'].mean_)]))]
    
    # Convert the pipeline to ONNX format
    onnx_model = convert_sklearn(pipe, initial_types=initial_type)
    
    # Save the ONNX model
    with open(onnx_filename, "wb") as f:
        f.write(onnx_model.SerializeToString())
        
# Convert the saved model to ONNX format
convert_to_onnx("data/models/model_education_loan.pkl", "data/models/model_education_loan.onnx")


/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.2 when using version 1.4.2. This might lead to break

KeyError: 'preprocessor'

In [70]:
import pickle
import numpy as np
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Example function to convert the Scikit-Learn model to ONNX
def convert_to_onnx(model_filename, onnx_filename):
    # Load the trained model
    with open(model_filename, "rb") as model_file:
        pipe = pickle.load(model_file)
    
    # Extract the classifier part of the pipeline
    classifier = pipe.named_steps['Classifier']
    
    # Define the initial types for ONNX conversion
    initial_type = [('input', FloatTensorType([None, 1]))]  # Adjust the size according to your input
    
    # Convert the classifier to ONNX format
    onnx_model = convert_sklearn(classifier, initial_types=initial_type)
    
    # Save the ONNX model
    with open(onnx_filename, "wb") as f:
        f.write(onnx_model.SerializeToString())

# Convert the saved model to ONNX format
convert_to_onnx("data/models/model_education_loan.pkl", "onnx_models/model_education_loan.onnx")


In [72]:
!mo --input_model onnx_models/model_education_loan.onnx --output_dir openvino_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
[ ERROR ]  -------------------------------------------------
[ ERROR ]  ----------------- INTERNAL ERROR ----------------
[ ERROR ]  Unexpected exception happened.
[ ERROR ]  Please contact Model Optimizer developers and forward the following information:
[ ERROR ]  Check 'false' failed at src/frontends/onnx/frontend/src/frontend.cpp:144:
FrontEnd API failed with GeneralFailure:
OpenVINO does not support the following ONNX operations: ai.onnx.ml.ZipMap, ai.onnx.ml.ArrayF